<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/recipe_to_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3siZmlsZV9kYXRhIjp7Im1pbWVfdHlwZSI6ImltYWdlL2pwZWciLCJ1cmwiOiJodHRwczovL3d3dy5nc3RhdGljLmNvbS9haXN0dWRpby9zYW1wbGUtaW1hZ2VzL2Nyb2lzc2FudC5qcGcifX0seyJ0ZXh0IjoiR2l2ZW4gdGhpcyBpbWFnZTpcXG5cXG4xLiBGaXJzdCwgZGVzY3JpYmUgdGhlIGltYWdlXFxuMi4gVGhlbiwgZGV0YWlsIHRoZSByZWNpcGUgdG8gYmFrZSB0aGlzIGl0ZW0gaW4gSlNPTiBmb3JtYXQuIEluY2x1ZGUgaXRlbSBuYW1lcyBhbmQgcXVhbnRpdGllcyBmb3IgdGhlIHJlY2lwZSJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IlRoZSBpbWFnZSBzaG93cyBhIGJhc2tldCBmdWxsIG9mIGZyZXNobHkgYmFrZWQgY3JvaXNzYW50cy4gVGhleSBhcmUgZ29sZGVuIGJyb3duIGFuZCBmbGFreSwgYW5kIHNvbWUgb2YgdGhlbSBoYXZlIGEgc2xpZ2h0bHkgYnVybnQgZWRnZS4gVGhleSBhcmUgcGlsZWQgaGlnaCBpbiB0aGUgYmFza2V0LCBhbmQgc29tZSBvZiB0aGVtIGFyZSBvdmVybGFwcGluZy4gVGhlIGJhY2tncm91bmQgaXMgYmx1cnJlZCwgYnV0IGl0IGFwcGVhcnMgdG8gYmUgYSBraXRjaGVuIGNvdW50ZXIgb3IgdGFibGUuXFxuXFxue1xcbiAgXFxcInRpdGxlXFxcIjogXFxcIkNyb2lzc2FudHNcXFwiLFxcbiAgXFxcImluZ3JlZGllbnRzXFxcIjogW1xcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQWxsLXB1cnBvc2UgZmxvdXJcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCI1MDBnXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiU2FsdFxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjEwZ1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIlN1Z2FyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMjBnXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiSW5zdGFudCB5ZWFzdFxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjdnXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiVW5zYWx0ZWQgYnV0dGVyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMjUwZ1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIk1pbGtcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIyNTBtbFxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIkVnZyB5b2xrXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMVxcXCJcXG4gICAgfVxcbiAgXSxcXG4gIFxcXCJpbnN0cnVjdGlvbnNcXFwiOiBbXFxuICAgIFxcXCJJbiBhIGxhcmdlIGJvd2wsIGNvbWJpbmUgdGhlIGZsb3VyLCBzYWx0LCBzdWdhciwgYW5kIHllYXN0LlxcXCIsXFxuICAgIFxcXCJBZGQgdGhlIG1pbGsgYW5kIG1peCB1bnRpbCBhIGRvdWdoIGZvcm1zLlxcXCIsXFxuICAgIFxcXCJLbmVhZCB0aGUgZG91Z2ggZm9yIDEwIG1pbnV0ZXMsIHVudGlsIGl0IGlzIHNtb290aCBhbmQgZWxhc3RpYy5cXFwiLFxcbiAgICBcXFwiUGxhY2UgdGhlIGRvdWdoIGluIGEgZ3JlYXNlZCBib3dsLCBjb3ZlciwgYW5kIGxldCBpdCByaXNlIGluIGEgd2FybSBwbGFjZSBmb3IgMSBob3VyLlxcXCIsXFxuICAgIFxcXCJSb2xsIG91dCB0aGUgZG91Z2ggaW50byBhIHJlY3RhbmdsZSBhYm91dCAxLzQgaW5jaCB0aGljay5cXFwiLFxcbiAgICBcXFwiU3ByZWFkIHRoZSBzb2Z0ZW5lZCBidXR0ZXIgZXZlbmx5IG92ZXIgdGhlIGRvdWdoLlxcXCIsXFxuICAgIFxcXCJGb2xkIHRoZSBkb3VnaCBpbiB0aGlyZHMsIGxpa2UgYSBsZXR0ZXIuXFxcIixcXG4gICAgXFxcIldyYXAgdGhlIGRvdWdoIGluIHBsYXN0aWMgd3JhcCBhbmQgcmVmcmlnZXJhdGUgZm9yIDMwIG1pbnV0ZXMuXFxcIixcXG4gICAgXFxcIlJlcGVhdCBzdGVwcyA3IGFuZCA4IHR3byBtb3JlIHRpbWVzLlxcXCIsXFxuICAgIFxcXCJSb2xsIG91dCB0aGUgZG91Z2ggaW50byBhIHJlY3RhbmdsZSBhYm91dCAxLzIgaW5jaCB0aGljay5cXFwiLFxcbiAgICBcXFwiQ3V0IHRoZSBkb3VnaCBpbnRvIHRyaWFuZ2xlcy5cXFwiLFxcbiAgICBcXFwiUm9sbCB1cCBlYWNoIHRyaWFuZ2xlLCBzdGFydGluZyBmcm9tIHRoZSB3aWRlIGVuZC5cXFwiLFxcbiAgICBcXFwiUGxhY2UgdGhlIGNyb2lzc2FudHMgb24gYSBiYWtpbmcgc2hlZXQgbGluZWQgd2l0aCBwYXJjaG1lbnQgcGFwZXIuXFxcIixcXG4gICAgXFxcIkNvdmVyIGFuZCBsZXQgdGhlbSByaXNlIGZvciAxIGhvdXIuXFxcIixcXG4gICAgXFxcIkJydXNoIHRoZSBjcm9pc3NhbnRzIHdpdGggZWdnIHlvbGsgYW5kIGJha2UgYXQgMzc1IGRlZ3JlZXMgRmFocmVuaGVpdCBmb3IgMTUtMjAgbWludXRlcywgb3IgdW50aWwgZ29sZGVuIGJyb3duLlxcXCJcXG4gIF0sXFxuICBcXFwibm90ZXNcXFwiOiBcXFwiWW91IGNhbiB1c2UgYSBzdGFuZCBtaXhlciB3aXRoIGEgZG91Z2ggaG9vayB0byBrbmVhZCB0aGUgZG91Z2guIE1ha2Ugc3VyZSB0aGUgYnV0dGVyIGlzIHNvZnRlbmVkIHRvIHJvb20gdGVtcGVyYXR1cmUgYmVmb3JlIHNwcmVhZGluZyBpdCBvbiB0aGUgZG91Z2guIFRoZSBjcm9pc3NhbnRzIHdpbGwgcmlzZSBiZXN0IGluIGEgd2FybSBwbGFjZSwgbGlrZSBhIHR1cm5lZC1vZmYgb3ZlbiB3aXRoIGEgYm93bCBvZiBob3Qgd2F0ZXIgaW5zaWRlLlxcXCJcXG59In1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJyZWNpcGUgb2YgYmlyeWFuaSBpIGpzb24gZm9ybWF0In1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiYGBganNvblxcbntcXG4gIFxcXCJ0aXRsZVxcXCI6IFxcXCJDaGlja2VuIEJpcnlhbmlcXFwiLFxcbiAgXFxcImluZ3JlZGllbnRzXFxcIjogW1xcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQ2hpY2tlblxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjEga2dcXFwiLFxcbiAgICAgIFxcXCJub3Rlc1xcXCI6IFxcXCJDdXQgaW50byBiaXRlLXNpemVkIHBpZWNlc1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIkJhc21hdGkgcmljZVxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjIgY3Vwc1xcXCIsXFxuICAgICAgXFxcIm5vdGVzXFxcIjogXFxcIlNvYWtlZCBpbiB3YXRlciBmb3IgMzAgbWludXRlc1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIk9uaW9uXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMlxcXCIsXFxuICAgICAgXFxcIm5vdGVzXFxcIjogXFxcIlNsaWNlZFxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIlRvbWF0b1xcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjJcXFwiLFxcbiAgICAgIFxcXCJub3Rlc1xcXCI6IFxcXCJQdXJlZWRcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJHaW5nZXItZ2FybGljIHBhc3RlXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMiB0YnNwXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiR3JlZW4gY2hpbGllc1xcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjQtNVxcXCIsXFxuICAgICAgXFxcIm5vdGVzXFxcIjogXFxcIlNsaXRcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJZb2d1cnRcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxIGN1cFxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIkdoZWVcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxLzIgY3VwXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiT2lsXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMiB0YnNwXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQmF5IGxlYWZcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIyXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQ2lubmFtb24gc3RpY2tcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQ2FyZGFtb20gcG9kc1xcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjRcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJDbG92ZXNcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCI0XFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQmxhY2sgcGVwcGVyY29ybnNcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxIHRzcFxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJuYW1lXFxcIjogXFxcIlR1cm1lcmljIHBvd2RlclxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjEgdHNwXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiUmVkIGNoaWxpIHBvd2RlclxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjEgdHNwXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIm5hbWVcXFwiOiBcXFwiQ3VtaW4gcG93ZGVyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMSB0c3BcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJDb3JpYW5kZXIgcG93ZGVyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMiB0c3BcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJTYWx0XFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwidG8gdGFzdGVcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwibmFtZVxcXCI6IFxcXCJGcmVzaCBjaWxhbnRyb1xcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjEvMiBjdXBcXFwiLFxcbiAgICAgIFxcXCJub3Rlc1xcXCI6IFxcXCJDaG9wcGVkIGZvciBnYXJuaXNoXFxcIlxcbiAgICB9XFxuICBdLFxcbiAgXFxcImluc3RydWN0aW9uc1xcXCI6IFtcXG4gICAgXFxcIk1hcmluYXRlIHRoZSBjaGlja2VuIHdpdGggeW9ndXJ0LCBnaW5nZXItZ2FybGljIHBhc3RlLCBncmVlbiBjaGlsaWVzLCB0dXJtZXJpYyBwb3dkZXIsIHJlZCBjaGlsaSBwb3dkZXIsIGN1bWluIHBvd2RlciwgY29yaWFuZGVyIHBvd2RlciwgYW5kIHNhbHQuIExldCBpdCBtYXJpbmF0ZSBmb3IgYXQgbGVhc3QgMzAgbWludXRlcy5cXFwiLFxcbiAgICBcXFwiSW4gYSBsYXJnZSBwb3QsIGhlYXQgZ2hlZSBhbmQgb2lsLiBBZGQgYmF5IGxlYWYsIGNpbm5hbW9uIHN0aWNrLCBjYXJkYW1vbSBwb2RzLCBjbG92ZXMsIGFuZCBibGFjayBwZXBwZXJjb3Jucy4gT25jZSB0aGV5IHN0YXJ0IHRvIHNwbHV0dGVyLCBhZGQgc2xpY2VkIG9uaW9ucyBhbmQgc2F1dMOpIHVudGlsIGdvbGRlbiBicm93bi5cXFwiLFxcbiAgICBcXFwiQWRkIHRoZSBtYXJpbmF0ZWQgY2hpY2tlbiBhbmQgY29vayB1bnRpbCBpdCB0dXJucyB3aGl0ZSBhbmQgaXMgYWxtb3N0IGZ1bGx5IGNvb2tlZC5cXFwiLFxcbiAgICBcXFwiQWRkIHRoZSB0b21hdG8gcHVyZWUgYW5kIGNvb2sgdW50aWwgdGhlIG9pbCBzZXBhcmF0ZXMgZnJvbSB0aGUgbWFzYWxhLlxcXCIsXFxuICAgIFxcXCJBZGQgdGhlIHNvYWtlZCBhbmQgZHJhaW5lZCBiYXNtYXRpIHJpY2UuIFN0aXIgd2VsbCBhbmQgYWRkIDIuNSBjdXBzIG9mIHdhdGVyLlxcXCIsXFxuICAgIFxcXCJCcmluZyB0aGUgbWl4dHVyZSB0byBhIGJvaWwsIHRoZW4gcmVkdWNlIGhlYXQgdG8gbG93LCBjb3ZlciwgYW5kIHNpbW1lciBmb3IgMTUtMjAgbWludXRlcyBvciB1bnRpbCB0aGUgcmljZSBpcyBjb29rZWQuXFxcIixcXG4gICAgXFxcIk9uY2UgY29va2VkLCBmbHVmZiB0aGUgYmlyeWFuaSB3aXRoIGEgZm9yayBhbmQgZ2FybmlzaCB3aXRoIGZyZXNoIGNpbGFudHJvLlxcXCIsXFxuICAgIFxcXCJTZXJ2ZSBob3Qgd2l0aCByYWl0YSBvciBzYWxhZC5cXFwiXFxuICBdXFxufVxcbmBgYCJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

Uploading: https://www.gstatic.com/aistudio/sample-images/croissant.jpg
[
    {
        "role": "user",
        "parts": [
            {
                "file_data": {
                    "mime_type": "image/jpeg",
                    "file_uri": "https://generativelanguage.googleapis.com/v1beta/files/sj1lu6zstwry"
                }
            },
            {
                "text": "Given this image:\\n\\n1. First, describe the image\\n2. Then, detail the recipe to bake this item in JSON format. Include item names and quantities for the recipe"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "The image shows a basket full of freshly baked croissants. They are golden brown and flaky, and some of them have a slightly burnt edge. They are piled high in the basket, and some of them are overlapping. The background is blurred, but it appears to be a kitchen counter or table.\\n\\n{\\n  \\\"title\\\": \\\"Croissants\\\"

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

This recipe gives you a basic outline for making chicken biryani. Feel free to adjust the ingredients and spices to your taste. For example, you can add other vegetables like potatoes, carrots, or peas, or you can use a different type of meat, such as lamb or beef. You can also adjust the level of spice by adding more or less red chili powder or green chilies.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 




Given this image:\n\n1. First, describe the image\n2. Then, detail the recipe to bake this item in JSON format. Include item names and quantities for the recipe 

-------------------------------------------------------------------------------- 

Role: model 

The image shows a basket full of freshly baked croissants. They are golden brown and flaky, and some of them have a slightly burnt edge. They are piled high in the basket, and some of them are overlapping. The background is blurred, but it appears to be a kitchen counter or table.\n\n{\n  \"title\": \"Croissants\",\n  \"ingredients\": [\n    {\n      \"name\": \"All-purpose flour\",\n      \"quantity\": \"500g\"\n    },\n    {\n      \"name\": \"Salt\",\n      \"quantity\": \"10g\"\n    },\n    {\n      \"name\": \"Sugar\",\n      \"quantity\": \"20g\"\n    },\n    {\n      \"name\": \"Instant yeast\",\n      \"quantity\": \"7g\"\n    },\n    {\n      \"name\": \"Unsalted butter\",\n      \"quantity\": \"250g\"\n    },\n    {\n  